<a href="https://colab.research.google.com/github/fabiobasson/my-repo/blob/main/oficios_qualidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Preciso criar um diretorio chamado laudo e jogar os oficios para dentro do mesmo (arquivos.pdf)
! mkdir -p laudo

from google.colab import files
uploaded = files.upload()

Saving Laudo_2628256.pdf to Laudo_2628256.pdf
Saving Laudo_2628563.pdf to Laudo_2628563.pdf
Saving Laudo_2787644.pdf to Laudo_2787644.pdf
Saving Laudo_2891654.pdf to Laudo_2891654.pdf
Saving Laudo_3014818.pdf to Laudo_3014818.pdf
Saving Laudo_3047648.pdf to Laudo_3047648.pdf
Saving Laudo_3069378.pdf to Laudo_3069378.pdf
Saving Laudo_3069866.pdf to Laudo_3069866.pdf


In [2]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# instalação de pacotes e bibliotecas

#!git clone https://www.github.com/camelot-dev/camelot
#!cd camelot
!pip install "camelot-py[cv]"

!apt-get install ghostscript

!pip install 'PyPDF2<3.0'
!pip install xlwt

!pip install pdfminer.six

import camelot
from pdfminer.high_level import extract_text

import pandas as pd
import io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.3/249.3 kB 20.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of camelot-py[cv] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono f

In [63]:
# Lendo documento .pdf e extraindo as tabela com a biblioteca Camelot

lista_tabelas = camelot.read_pdf('Laudo_2628256.pdf',header=None,index=False,encoding="utf-8",pages="all")

# Criando Dataframe após extração das tabelas do arquivo .pdf

lista_tabelas = lista_tabelas[0].df[[1,0,2]].reset_index(drop=True)

# Extraindo texto do documento com a biblioteca Extract_text

texto  = extract_text('Laudo_2628256.pdf', 'rb')

# Criando Dataframe após extração das tabelas do arquivo .pdf

#texto = pd.read_fwf(io.StringIO(texto))

#texto = texto[16:22]


In [56]:
lista_tabelas

,1,0,2
0,,3. RELAÇÃO DE DADOS RECEBIDOS,
1,Norma de Referência:,Tipos de Dados:,Situação dos Dados no BDEP*:
2,RANP n° 880/2022,3.1. Arquivos Auxiliares - Esquemas de Ferrame...,NÃO CONFORME
3,RANP n° 880/2022,3.2. Arquivos Auxiliares - Dados Direcionais,APROVADO
4,RANP n° 880/2022,3.3. Arquivos Auxiliares - Relatório de Campo ...,
5,RANP n° 880/2022,3.4. Perfis Durante a Perfuração,NÃO CONFORME
6,RANP n° 880/2022,3.5. Perfis Convencionais,APROVADO
7,RANP n° 880/2022,3.6. Perfis Especiais - Teste de Formação a Cabo,APROVADO
8,RANP n° 880/2022,3.7. Perfis Especiais - Sísmica de Poço,
9,RANP n° 880/2022,3.8. Perfis Especiais - Amostrador Lateral,APROVADO


In [6]:
# Iniciando o ETL da extração das tabelas

# remover as linhas e colunas desnecessárias a extração das informações

lista_tabelas = lista_tabelas.drop(0, axis='index')
lista_tabelas = lista_tabelas.drop(1, axis='index')

# Renomear as colunas para o padrão do documento

lista_tabelas.rename(columns = {1:'Norma Referencia',0:'Tipo Dados',2:'Situacao BDIEP'}, inplace = True)
lista_tabelas

# Iniciando o ETL da extração do texto

# remover linhas e colunas desnecessárias a extração das informações

# Renomear colunas para o padrão do documento

texto.rename(columns = {'Agencia Nacional de Petróleo, Gás Natura e Biocombustíveis':' CABEÇALHO'}, inplace = True)
texto = texto.T
texto.rename(columns = {16:'Nº do Laudo de Reprovação', 17:'Data da Avaliação',18:'Nome do Poço',19:'Nº Cadastro do Poço',20:'Conclusão do Poço',21:'Empresa Operadora'}, inplace = True)


In [7]:
texto

,Nº do Laudo de Reprovação,Data da Avaliação,Nome do Poço,Nº Cadastro do Poço,Conclusão do Poço,Empresa Operadora
CABEÇALHO,2211.1.0210,18/11/2022,7-BUZ-33-RJS,74316029460,13/03/2020,Petrobras


In [ ]:
# Concatenando os dataframes (texto e tabelas )

concatenar = pd.concat([texto,lista_tabelas],ignore_index=True)
concatenar.shape
concatenar

,Nº do Laudo de Reprovação,Data da Avaliação,Nome do Poço,Nº Cadastro do Poço,Conclusão do Poço,Empresa Operadora,Norma Referencia,Tipo Dados,Situacao BDIEP
0,2211.1.0210,18/11/2022,7-BUZ-33-RJS,74316029460,13/03/2020,Petrobras,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.1. Arquivos Auxiliares - Esquemas de Ferrame...,NÃO CONFORME
2,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.2. Arquivos Auxiliares - Dados Direcionais,APROVADO
3,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.3. Arquivos Auxiliares - Relatório de Campo ...,
4,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.4. Perfis Durante a Perfuração,NÃO CONFORME
5,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.5. Perfis Convencionais,APROVADO
6,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.6. Perfis Especiais - Teste de Formação a Cabo,APROVADO
7,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.7. Perfis Especiais - Sísmica de Poço,
8,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.8. Perfis Especiais - Amostrador Lateral,APROVADO
9,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.9. Perfil Digital Processado,NÃO CONFORME


In [ ]:
# Saida do dataframe para um planilha excel
concatenar.to_excel('file_name.xls')

In [8]:
import re
import os
# Caminho da pasta
path = r'/content/'
# Percorrer somente pelos arquivos .pdf
for file in os.listdir(path):                  
    if re.search('pdf', file):                          
        print(file)

Laudo_2628563.pdf
Laudo_2628256.pdf
Laudo_3014818.pdf
Laudo_2787644.pdf
Laudo_3069866.pdf
Laudo_3047648.pdf
Laudo_3069378.pdf
Laudo_2891654.pdf


In [11]:
import glob
filename=glob.glob(path + "/*.pdf")
lista_arquivos = []

for i in filename:
   lista_arquivos.append(pd.read_csv(i))

full = pd.concat(lista_arquivos,ignore_index=True)

UnicodeDecodeError: ignored

In [17]:
# Extraindo o texto do arquivo .pdf

from PyPDF2 import PdfFileReader


def text_extractor(path):
     with open(path, 'rb') as arquivo:
      pdf = PdfFileReader(arquivo)

      # get the first page
      page = pdf.getPage(1)
      #print(page)
      #print('Page type: {}'.format(str(type(page))))
      texto = page.extractText()
      print(texto)

if __name__ == '__main__':
   path = r"/content/Laudo_2628256.pdf"
   text_extractor(path)


Itens: Não Conformidades:5. NÃO CONFORMIDADES REFERENTES À REMESSA DE DADOS , À ESTRUTURA DE DIRETÓRIOS E AO FORMATO E 
NOMENCLATURA DOS ARQUIVOS
5.1. Remessa de Dados
5.2. Estrutura de Diretórios
5.3. Formato / Nomenclatura
6. NÃO CONFORMIDADES REFERENTES AOS RELATÓRIOS E ARQUIVOS AUXILIARES
Não Conformidades: Itens:
., Ausência do valor associado ao Nome do Poço. Valor esperado: 7-BUZ-33-RJS. 
Ausência do valor associado ao Cadastro de Poço. Valor esperado :74316029460.6.1. Esquemas de Ferramentas
6.2. Dados Direcionais
6.3. Rel. Sísmica de Poço
7. NÃO CONFORMIDADES REFERENTES AO CABEÇALHO DE PERFIS DE POÇO (DURANTE A PERFURAÇÃO , 
CONVENCIONAIS, ESPECIAIS, COMPOSTO E DE ACOMPANHAMENTO GEOLÓGICO )
Itens: Não Conformidades:
7.1. Identificação do poço, 
bloco/campo e operador
7.2. Informações e 
Referências de profundidades
7.3. Identificação do perfil
7.4. Coordenadas 
(geográficas/ retangulares)
7.5. Informação de fluido de 
perfuração
7.6. Informação de 
revestimento
7.7. Informação

In [127]:
import PyPDF2

pdf_file = open('Laudo_3047648.pdf','rb')

#dados_pdf = PyPDF2.PdfFileReader(pdf_file)
#num_pages = dados_pdf.getNumPages()

with open('Laudo_3047648.pdf', 'rb') as arquivo:
    leitor_pdf = PyPDF2.PdfFileReader(arquivo)

    # Obtém o número total de páginas do PDF
    total_paginas = leitor_pdf.numPages

    for numero_pagina in range(total_paginas):
        pagina = leitor_pdf.getPage(numero_pagina)
        texto = pagina.extractText()
        #texto = pd.read_fwf(io.StringIO(texto))
        # Faça algo com o texto extraído da página
        print(f"Conteúdo da página {numero_pagina + 1}:")
        print(texto)
        print("\n")
        f = open("texto.txt", "a+")
        f.write("\n" + texto)
        f.close()
    
#display(texto_pagina0.split("\n"))


Conteúdo da página 1:
Agencia Nacional de Petróleo, Gás Natura e Biocombustíveis
Superintendência de Dados Técnicos
Banco de Dados de Exploração e Produção
Laudo de Avaliação de Dados de Poço
2305.1.0099 7-MRO-6DB-RJS 74316029648 23/02/2021 Petrobras1.1. Nº do Laudo 
de Reprovação:1.2. Data da 
Avaliação :1.3. Nome do 
Poço:1.4. Nº Cadastro do 
Poço:1.5. Conclusão do 
Poço:1.6. Empresa 
Operadora:1. CABEÇALHO
08/05/2023
2.3. Código de Cadastro da(s) 
Mídia(s) no BDEP:2.2. Data de Protocolo da(s) 
Remessa(s):2.1. Número de Protocolo da(s) 
Remessa(s):2. CONTROLE DE REMESSAS (USO INTERNO)
48610.202029/2021-13 370_2021 14/05/2021
48610.202029/2021-13 385_2021 17/05/2021
*Legenda da Situação dos Dados de Poço no BDEP:
NÃO RECEBIDO = Conjunto de dados adquiridos pela operadora, conforme Notificação de Perfilagem Realizada (NPR), mas não 
remetidos ao BDEP;
NÃO CONFORME = Conjunto de dados avaliados como não conformes. As não conformidades estão descritas neste laudo;
APROVADO = Conjunto de 

In [128]:
type (texto)

str

In [129]:
search_for = "NÃO CONFORMIDADES" # What to search in the log lines...
line_num = 0 # Line number
lines_found = 0 # Quantity of lines found...
out_file = "saida.txt"
in_file = "texto.txt"

with open(out_file, 'w') as out_file:
    with open(in_file, "r") as in_file:
        for line in in_file:
            line_num += 1
            if search_for in line:
                lines_found += 1
                print("Found '{}' in line {}...".format(search_for, line_num))
                out_file.write(line)

        print("Found {} lines...".format(lines_found))

Found 'NÃO CONFORMIDADES' in line 50...
Found 'NÃO CONFORMIDADES' in line 56...
Found 'NÃO CONFORMIDADES' in line 62...
Found 'NÃO CONFORMIDADES' in line 88...
Found 'NÃO CONFORMIDADES' in line 113...
Found 5 lines...


In [ ]:
descproblema = ['Valor associado a menmonicos'],['Dados nao remetidos ao BDEP'],['Ausência de informação de perflagem conforme NPR '],['Valor inválido associado a mnemônicos '],['Cabeçalho '],['Valor inválido associado a mnemônicos'],['Ausência de informação de perflagem conforme NPR']
df4 = pd.DataFrame(descproblema,columns=['Descrição do Problema'])

df4 = df4.reset_index(drop=True)
df4.info()
#df4.rename(columns = {'16':'team_name', '17':'points_scored'}, inplace = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 1 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Descrição do Problema  7 non-null      object
dtypes: object(1)
memory usage: 184.0+ bytes


In [ ]:
!pip install tabula-py
!pip install tabulate

from tabula import read_pdf
from tabulate import tabulate


# Read the only the page n°0 of the file
texto_pesquisa = read_pdf('Laudo_2628256.pdf',pages = 1,multiple_tables = True, stream = True, encoding="utf-8")

# Transform the result into a string table format
tabela_pesquisa = tabulate(texto_pesquisa)

# Transform the table into dataframe
df_pesq = pd.read_fwf(io.StringIO(tabela_pesquisa))
df_pesq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,---------------------------------,----------------------------------,-----------------------------------,-------------------------------,-------------------------------.1,-------------------------------.2,----------------------------------------------------------,-------------------------------------------------------,-------------------------------------------------------.1
0,0 1.1. No do Laudo,0 1.2. Data da,0 1.3. Nome do,0 1.4. No Cadastro do,0 1.5. Conclusão do,0 1.6. Empresa,NaN,NaN,NaN
1,1 de Reprovação:,1 Avaliação:,1 Poço:,1 Poço:,1 Poço:,1 Operadora:,NaN,NaN,NaN
2,2 2211.1.0210,2 18/11/2022,2 7-BUZ-33-RJS,2 74316029460,2 13/03/2020,2 Petrobras,NaN,NaN,NaN
3,"Name: 1. CABEÇALHO, dtype: object","Name: Unnamed: 0, dtype: object","Name: Unnamed: 1, dtype: object","Name: Unnamed: 2, dtype: object","Name: Unnamed: 3, dtype: object","Name: Unnamed: 4, dtype: object",NaN,NaN,NaN
4,NaN,0 2.3. Código de Cadastro da(s),NaN,NaN,NaN,NaN,0 2.1. Número de Protocolo da(s) 2.2. Data ...,NaN,NaN
5,NaN,1 Mídia(s) no BDEP:,NaN,NaN,NaN,NaN,1 Remessa(s): Rem...,NaN,NaN
6,NaN,"Name: Unnamed: 0, dtype: object",NaN,NaN,NaN,NaN,"Name: 2. CONTROLE DE REMESSAS (USO INTERNO), d...",NaN,NaN
7,NaN,0 Situação dos Dados no BDEP*:,NaN,NaN,NaN,NaN,NaN,0 Tipos de Dados: Norma de Re...,NaN
8,NaN,1 NÃO CONFORME,NaN,NaN,NaN,NaN,NaN,1 3.1. Arquivos Auxiliares - Esquemas de F...,NaN
9,NaN,2 APROVADO,NaN,NaN,NaN,NaN,NaN,2 3.2. Arquivos Auxiliares - Dados Direcio...,NaN


In [ ]:
print(tables[0].parsing_report)
>>> {'accuracy': 100.0, 'whitespace': 0.0, 'order': 1, 'page': 1}
for i in range(tables.n):
    print(tables[i].parsing_report)
>>> {'accuracy': 100.0, 'whitespace': 0.0, 'order': 1, 'page': 1}
>>> {'accuracy': 100.0, 'whitespace': 0.0, 'order': 2, 'page': 1}

print(tables[0])

for i in range(tables.n):
    print(tables[i])

{'accuracy': 100.0, 'whitespace': 37.5, 'order': 1, 'page': 1}
{'accuracy': 100.0, 'whitespace': 37.5, 'order': 1, 'page': 1}
<Table shape=(14, 4)>
<Table shape=(14, 4)>
